In [1]:
import pandas as pd
import string
from Preprocessing import preprocessing

In [2]:
# delete output file if already exists
!rm data/processed/output_POS.txt
# call preprocessing to initiate processing all the books
preprocessing()

Done processing: Devices And Desires
Done processing: The Sandcastle
Done processing: Destination Unknown
Done processing: Elephants Can Remember
Done processing: Death Comes To Pemberley
Done processing: Cover Her Face
Done processing: Jacksons Dilemma
Done processing: And Then There Were None
Done processing: The Black Prince


In [3]:
book_info_df = pd.read_csv("data/processed/output_POS.txt", 
                           delimiter='|',
                           index_col=False,
                           quoting=3, 
                           encoding='utf-8')

print(book_info_df.shape)
book_info_df

(9, 71)


,Author,Title,book_name,total_words,avg_sentence_size,!,#,"""",%,$,...,VBD,VBG,VBP,VBN,WDT,VBZ,WRB,WP$,WP,Unnamed: 70
0,P.D. James,Devices And Desires,DevicesAndDesires,157206,14.981988,0.001293,0.000086,0,0.000000,0.040767,...,0.067373,0.015981,0.022169,0.022631,0.003464,0.012071,0.005285,0.000094,0.004886,NaN
1,Iris Murdoch,The Sandcastle,TheSandcastle,113888,13.673670,0.016663,0.000000,0,0.000000,0.041840,...,0.083745,0.018261,0.014774,0.020327,0.005340,0.007362,0.003926,0.000117,0.004087,NaN
2,Agatha Christie,Destination Unknown,DestinationUnknown,60461,10.251102,0.017421,0.000711,0,0.000000,0.017510,...,0.062762,0.012496,0.030155,0.017879,0.002633,0.016213,0.004376,0.000039,0.006028,NaN
3,Agatha Christie,Elephants Can Remember,ElephantsCanRemember,60024,11.163102,0.003683,0.000000,0,0.000000,0.018604,...,0.064873,0.010579,0.040799,0.017196,0.004067,0.015221,0.005008,0.000052,0.009167,NaN
4,P.D. James,Death Comes To Pemberley,DeathComesToPemberley,90969,19.236414,0.002227,0.000000,0,0.000000,0.017722,...,0.067453,0.015016,0.012460,0.032502,0.005054,0.010069,0.005618,0.000185,0.004510,NaN
5,P.D. James,Cover Her Face,CoverHerFace,77527,13.320790,0.007964,0.000263,0,0.000875,0.037455,...,0.073576,0.014477,0.015781,0.026823,0.003134,0.010517,0.004963,0.000145,0.005231,NaN
6,Iris Murdoch,Jacksons Dilemma,JacksonsDilemma,88190,17.110982,0.032038,0.005796,0,0.000000,0.073758,...,0.076244,0.021844,0.019245,0.021162,0.002862,0.008986,0.006288,0.000082,0.005434,NaN
7,Agatha Christie,And Then There Were None,AndThenThereWereNone,52467,8.815020,0.027480,0.002574,0,0.000209,0.074301,...,0.073138,0.013511,0.017264,0.018840,0.001952,0.013285,0.003753,0.000030,0.005569,NaN
8,Iris Murdoch,The Black Prince,TheBlackPrince,132907,12.994427,0.016602,0.007218,0,0.000000,0.028103,...,0.057163,0.017470,0.032650,0.016863,0.003349,0.014710,0.005086,0.000054,0.004866,NaN
